In [1]:
import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

In [2]:
"""Build the pure visual model"""
from mvcl.percept.metanet import MetaNet
from mvcl.model import MetaVisualLearner, config
from mvcl.custom import ObjectAffinityFeatures

#affinity = ObjectAffinityFeatures(3, 100)
metanet = MetaVisualLearner(None, config)

metanet.add_affinities(["texture", "color", "category"])

[Taichi] version 1.7.0, llvm 15.0.7, commit 7b58b0ff, osx, python 3.9.13


[I 04/09/24 07:47:38.614 2883864] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [5]:
B, C, W, H = (1, 3, 64, 64)
fake_ims = torch.randn([B,C,W,H])
targets = torch.randn([B,W,H,1])

#metanet.eval()
metanet.freeze_components()
outputs=metanet.calculate_object_affinity(
    fake_ims,
    targets,
    working_resolution=(W,H),verbose=True)
print(outputs["affinity"].shape)
print("done")

torch.Size([1, 3, 4096, 58])
torch.Size([1, 4096, 58, 64]) torch.Size([1, 3, 64])
attn torch.Size([1, 3, 4096, 58]) tensor(1., grad_fn=<MaxBackward1>) tensor(4.0365e-13, grad_fn=<MinBackward1>)
torch.Size([1, 4096, 58]) tensor(0.1279, grad_fn=<MaxBackward1>) tensor(-0.1469, grad_fn=<MinBackward1>)
torch.Size([1, 4096, 58])
done


In [4]:
"""Write the code for the adaptive object affinity calculator"""
def calculate_object_affinity(self):
    """step 1: calculate the attention based on the component affinity key"""
    
    """step 2: calculate something real"""
    return 